In [25]:
import json
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

In [48]:
# Function to initialize the webdriver
def init_driver():
    driver = webdriver.Chrome()
    driver.get('https://mcd-demo-client.mycareerdreams.com/')
    return driver


In [55]:
# Function to fill the registration form with data from JSON
def fill_registration_form(driver, data):
    try:
        name_input = driver.find_element(By.XPATH, "//input[@id='name_input']")
        username_input = driver.find_element(By.XPATH, "//input[@id='username_input']")
        email_input = driver.find_element(By.XPATH, "//input[@id='email_input']")
        password_input = driver.find_element(By.XPATH, "//input[@id='pwd_input']")
        confirm_password_input = driver.find_element(By.XPATH, "//input[@id='pwd_confirm_input']")
        institute_code_input = driver.find_element(By.XPATH, "//input[@id='institute_code_input']")
        country_dropdown = Select(driver.find_element(By.XPATH, "//select[@id='country_id']"))
        state_dropdown = Select(driver.find_element(By.XPATH, "//select[@id='state_id']"))
        city_dropdown = Select(driver.find_element(By.XPATH, "//select[@id='city_id']"))
        register_button = driver.find_element(By.XPATH, "//button[@id='register-btn']")

        # Enter data into the form
        name_input.clear()
        name_input.send_keys(data['name'])
        username_input.clear()
        username_input.send_keys(data['username'])
        email_input.clear()
        email_input.send_keys(data['email'])
        password_input.clear()
        password_input.send_keys(data['password'])
        confirm_password_input.clear()
        confirm_password_input.send_keys(data['confirmPassword'])
        institute_code_input.clear()
        institute_code_input.send_keys(data['instituteCode'])

        # Select dropdown options
        country_dropdown.select_by_visible_text(data['country'])
        
        time.sleep(5)
        # Wait for the state dropdown options to be visible
        state_dropdown_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, "state_id"))
        )
        state_dropdown = Select(state_dropdown_element)
        state_dropdown.select_by_visible_text(data['state'])
        
        time.sleep(5)
        # Wait for the city dropdown options to be enabled
        city_dropdown_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "city_id"))
        )
        city_dropdown = Select(city_dropdown_element)
        city_dropdown.select_by_visible_text(data['city'])

        # Check for specific error messages
        error_messages = {
            "email_format": "Invalid Email",
            "password_format": "Must contain 8 characters including 1 uppercase, 1 lowercase, 1 number and 1 special case character.",
            "password_match": "Password doesn't match"
        }
        errors_found = []

        try:
            email_error = driver.find_element(By.XPATH, "//p[@id='error-email-format' and contains(@style, 'display: block;')]")
            errors_found.append(error_messages["email_format"])
        except:
            pass

        try:
            password_format_error = driver.find_element(By.XPATH, "//p[@id='error-pwd-format' and contains(@style, 'display: block;')]")
            errors_found.append(error_messages["password_format"])
        except:
            pass

        try:
            password_match_error = driver.find_element(By.XPATH, "//p[@id='error-pwd' and text()=\"Password doesn't match\"]")
            errors_found.append(error_messages["password_match"])
        except:
            pass

        if errors_found:
            return "Errors found: " + ", ".join(errors_found)

        # Submit the form
        register_button.click()

        return "Form submitted"
    except Exception as e:
        return str(e)

In [56]:
# Function to append test results to a JSON file
def append_test_results(file_path, data):
    try:
        with open(file_path, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []

    existing_data.append(data)

    with open(file_path, 'w') as file:
        json.dump(existing_data, file, indent=4)


In [57]:
# Function to run a single test case
def run_test_case(data, result_file):
    driver = init_driver()
    print(f"Running test case: {data['title']}")
    result = fill_registration_form(driver, data)
    data['status'] = result

    if "Errors found" not in result:
        try:
            popup = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "swal2-html-container")))
            data['popup_message'] = popup.text
        except:
            data['popup_message'] = "No popup message"
    else:
        data['popup_message'] = result

    # Append the result to the result file
    append_test_results(result_file, data)

    driver.quit()

In [60]:
# Test cases
test_case_1 = {
    "title": "Valid Registration",
    "name": "John Doe",
    "username": "johndoe123",
    "email": "john.doe@example.com",
    "password": "Test@123",
    "confirmPassword": "Test@123",
    "instituteCode": "ABC123",
    "country": "Pakistan",
    "state": "Punjab",
    "city": "Lahore"
}

test_case_2 = {
    "title": "Invalid Email Format",
    "name": "Alice Smith",
    "username": "alice123",
    "email": "invalidemail",
    "password": "Test@123",
    "confirmPassword": "Test@123",
    "instituteCode": "DEF456",
    "country": "Pakistan",
    "state": "Sindh",
    "city": "Karachi"
}

test_case_3 = {
    "title": "Password Mismatch",
    "name": "Bob Johnson",
    "username": "bob456",
    "email": "bob@example.com",
    "password": "Test@123",
    "confirmPassword": "Mismatch@123",
    "instituteCode": "GHI789",
    "country": "Pakistan",
    "state": "Khyber Pakhtunkhwa",
    "city": "Peshawar"
}

test_case_4 = {
    "title": "Empty Fields",
    "name": "",
    "username": "",
    "email": "",
    "password": "",
    "confirmPassword": "",
    "instituteCode": "",
    "country": "Pakistan",
    "state": "Punjab",
    "city": "Lahore"
}

test_case_5 = {
    "title": "Username Already Taken",
    "name": "David Brown",
    "username": "johndoe123",  # Existing username used intentionally
    "email": "david@example.com",
    "password": "Test@123",
    "confirmPassword": "Test@123",
    "instituteCode": "JKL012",
    "country": "Pakistan",
    "state": "Sindh",
    "city": "Karachi"
}

result_file = 'mcd_test_results.json'

In [66]:
test_case_6 = {
    "title": "Missing Required Fields",
    "name": "Jane Doe",
    "username": "janedoe123",
    "email": "",
    "password": "Test@123",
    "confirmPassword": "Test@123",
    "instituteCode": "",
    "country": "Pakistan",
    "state": "Sindh",
    "city": "Karachi"
}


In [67]:
test_case_7 = {
    "title": "Invalid Username Format",
    "name": "Charlie Brown",
    "username": "charlie@brown",
    "email": "charlie.brown@example.com",
    "password": "Test@123",
    "confirmPassword": "Test@123",
    "instituteCode": "MNO345",
    "country": "Pakistan",
    "state": "Punjab",
    "city": "Lahore"
}


In [68]:
test_case_8 = {
    "title": "Short Password",
    "name": "Emily White",
    "username": "emilywhite",
    "email": "emily.white@example.com",
    "password": "T@1a",
    "confirmPassword": "T@1a",
    "instituteCode": "PQR678",
    "country": "Pakistan",
    "state": "Khyber Pakhtunkhwa",
    "city": "Peshawar"
}


In [69]:
test_case_9 = {
    "title": "Long Username",
    "name": "Michael Green",
    "username": "michaelgreen12345678901234567890",
    "email": "michael.green@example.com",
    "password": "Test@123",
    "confirmPassword": "Test@123",
    "instituteCode": "STU901",
    "country": "Pakistan",
    "state": "Balochistan",
    "city": "Quetta"
}


In [70]:
test_case_10 = {
    "title": "Special Characters in Name",
    "name": "John*Doe",
    "username": "johndoe456",
    "email": "john.doe@example.com",
    "password": "Test@123",
    "confirmPassword": "Test@123",
    "instituteCode": "VWX234",
    "country": "Pakistan",
    "state": "Islamabad",
    "city": "Islamabad"
}


In [61]:
# Run individual test cases
run_test_case(test_case_1, result_file)


Running test case: Valid Registration


In [62]:
run_test_case(test_case_2, result_file)


Running test case: Invalid Email Format


In [63]:
run_test_case(test_case_3, result_file)


Running test case: Password Mismatch


In [64]:
run_test_case(test_case_4, result_file)


Running test case: Empty Fields


In [65]:
run_test_case(test_case_5, result_file)

Running test case: Username Already Taken


In [71]:
run_test_case(test_case_6,result_file)

Running test case: Missing Required Fields


In [72]:
run_test_case(test_case_7,result_file)

Running test case: Invalid Username Format


In [73]:
run_test_case(test_case_8,result_file)

Running test case: Short Password


In [74]:
run_test_case(test_case_9,result_file)

Running test case: Long Username


In [75]:
run_test_case(test_case_10,result_file)

Running test case: Special Characters in Name


# Login Page

In [76]:
# Function to initialize the webdriver
def init_driver():
    driver = webdriver.Chrome()
    driver.get('https://mcd-demo-client.mycareerdreams.com/')
    return driver

In [77]:
# Function to perform login testing
def login_test(driver, data):
    try:
        # Click the Log In tab
        login_tab = driver.find_element(By.XPATH, "//a[@href='#login']")
        login_tab.click()
        
        # Wait for the login form to be visible
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, "login"))
        )

        # Locate form elements
        email_input = driver.find_element(By.XPATH, "//input[@id='email_login_txt']")
        password_input = driver.find_element(By.XPATH, "//input[@id='pwd_login_txt']")
        institute_code_input = driver.find_element(By.XPATH, "//input[@id='institute_code']")
        login_button = driver.find_element(By.XPATH, "//button[@id='form_login_btn']")

        # Enter data into the form
        email_input.clear()
        email_input.send_keys(data['email'])
        password_input.clear()
        password_input.send_keys(data['password'])
        institute_code_input.clear()
        institute_code_input.send_keys(data['instituteCode'])

        # Submit the form
        login_button.click()

        # Wait for potential error message or successful login indication
        try:
            error_message = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.ID, "login-error-msg"))
            )
            return error_message.text
        except:
            return "Login successful or no error message displayed"
    
    except Exception as e:
        return str(e)


In [78]:
# Function to append test results to a JSON file
def append_test_results(file_path, data):
    try:
        with open(file_path, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []

    existing_data.append(data)

    with open(file_path, 'w') as file:
        json.dump(existing_data, file, indent=4)

In [79]:
# Function to run a single login test case
def run_login_test_case(data, result_file):
    driver = init_driver()
    print(f"Running login test case: {data['title']}")
    result = login_test(driver, data)
    data['status'] = result

    # Append the result to the result file
    append_test_results(result_file, data)

    driver.quit()

In [80]:
# Login test cases
login_test_case_1 = {
    "title": "Valid Login",
    "email": "john.doe@example.com",
    "password": "Test@123",
    "instituteCode": "ABC123"
}

login_test_case_2 = {
    "title": "Invalid Email Format",
    "email": "invalidemail",
    "password": "Test@123",
    "instituteCode": "ABC123"
}

login_test_case_3 = {
    "title": "Incorrect Password",
    "email": "john.doe@example.com",
    "password": "WrongPassword123",
    "instituteCode": "ABC123"
}

login_test_case_4 = {
    "title": "Empty Fields",
    "email": "",
    "password": "",
    "instituteCode": ""
}

login_test_case_5 = {
    "title": "Nonexistent User",
    "email": "nonexistent@example.com",
    "password": "Test@123",
    "instituteCode": "XYZ789"
}

result_file = 'login_test_results.json'



In [81]:
# Run individual login test cases
run_login_test_case(login_test_case_1, result_file)


Running login test case: Valid Login


In [82]:
run_login_test_case(login_test_case_2, result_file)


Running login test case: Invalid Email Format


In [83]:
run_login_test_case(login_test_case_3, result_file)


Running login test case: Incorrect Password


In [84]:
run_login_test_case(login_test_case_4, result_file)


Running login test case: Empty Fields


In [85]:
run_login_test_case(login_test_case_5, result_file)

Running login test case: Nonexistent User
